<a href="https://colab.research.google.com/github/khushiisaxena/Artificial-Intelligence/blob/main/AI_Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Que 1: Marble_Solitaire

In [ ]:
# Import necessary libraries for all three problem statements
import numpy as np
from time import time
import random
from queue import PriorityQueue

In [ ]:
# create a node class
class Node:
    def __init__(self, parent, state, path_cost, heuristic_cost, action=None):
        self.parent = parent
        self.state = state
        self.action = action
        self.path_cost = path_cost
        self.heuristic_cost = heuristic_cost
        self.cost = path_cost + heuristic_cost

    def __hash__(self):
        return hash(str(self.state.flatten()))

    def __str__(self):
        return str(self.state)

    def __ne__(self, other):
        return hash(''.join(self.state.flatten())) != hash(''.join(other.state.flatten()))

    def __eq__(self, other):
        return hash(''.join(self.state.flatten())) == hash(''.join(other.state.flatten()))

# create a priority queue class
class PriorityQueue:
    def __init__(self):
        self.queue = []
        self.hashes = {}

    def push(self, node):
        node_hash = hash(node)
        if node_hash not in self.hashes:
            self.hashes[node_hash] = 1
            self.queue.append(node)

    def pop(self):
        min_cost_index = self._find_min_cost_index()
        return self.queue.pop(min_cost_index)

    def _find_min_cost_index(self):
        min_cost = 10**18
        min_index = -1
        for i, node in enumerate(self.queue):
            if node.cost < min_cost:
                min_cost = node.cost
                min_index = i
        return min_index

    def is_empty(self):
        return not bool(self.queue)

    def __str__(self):
        return str([node.state for node in self.queue])

    def __len__(self):
        return len(self.queue)

# create a class for environment
class Environment:
    def __init__(self, start_state=None, goal_state=None):
        self.actions = [1, 2, 3, 4]  # 1 - Up, 2 - Down, 3 - Right, 4 - Left
        self.goal_state = goal_state if goal_state is not None else self.generate_goal_state()
        self.start_state = start_state if start_state is not None else self.generate_start_state()

    def generate_state(self, center_value):
        state = np.zeros((7, 7))
        state[[0, 1, 5, 6], :] = -1
        state[:, [0, 1, 5, 6]] = -1
        state[2:5, 2:5] = 1
        state[3, 3] = center_value
        return state

    def generate_start_state(self):
        return self.generate_state(0)

    def generate_goal_state(self):
        return self.generate_state(1)

    def get_start_state(self):
        return self.start_state

    def get_goal_state(self):
        return self.goal_state

    def get_next_states(self, state):
        new_states = []
        spaces = [(i, j) for i in range(7) for j in range(7) if state[i][j] == 0]
        for x, y in spaces:
            new_states.extend(self.get_new_states_for_space(state, x, y))
        return new_states

    def get_new_states_for_space(self, state, x, y):
        new_states = []
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
        for dx, dy in directions:
            if self.can_move(state, x, y, dx, dy):
                new_state = state.copy()
                new_state[x][y] = 1
                new_state[x + dx][y + dy] = 0
                new_state[x + 2 * dx][y + 2 * dy] = 0
                action = f'({x + 2 * dx}, {y + 2 * dy}) -> ({x}, {y})'
                new_states.append((new_state, action))
        return new_states

    def can_move(self, state, x, y, dx, dy):
        return (0 <= x + 2 * dx < 7 and 0 <= y + 2 * dy < 7 and
                state[x + dx][y + dy] == 1 and state[x + 2 * dx][y + 2 * dy] == 1)

    def reached_goal(self, state):
        return np.array_equal(state, self.goal_state)

In [ ]:
import time

class Player1:
    def __init__(self, env, heuristic):
        self.frontier = PriorityQueue()
        self.explored = {}
        self.env = env
        self.heuristic = heuristic
        self.start_state = self.env.get_start_state()
        self.goal_state = self.env.get_goal_state()
        self.goal_node = None

    def run(self):
        self.initialize_frontier()
        start_time = time.time()
        while not self.frontier.is_empty():
            current_node = self.frontier.pop()
            if self.is_explored(current_node):
                continue
            self.add_to_explored(current_node)
            if self.env.reached_goal(current_node.state):
                print("Reached goal!")
                self.goal_node = current_node
                break
            self.expand_frontier(current_node)
        end_time = time.time()
        print(end_time - start_time)
        return end_time - start_time

    def initialize_frontier(self):
        initial_node = Node(parent=None, state=self.start_state, path_cost=0, heuristic_cost=0)
        self.frontier.push(initial_node)

    def is_explored(self, node):
        return hash(node) in self.explored

    def add_to_explored(self, node):
        self.explored[hash(node)] = node

    def expand_frontier(self, current_node):
        next_states = self.env.get_next_states(current_node.state)
        for state in next_states:
            heuristic_cost = self.heuristic(state[0])
            node = Node(parent=current_node, state=state[0], path_cost=current_node.path_cost+1, heuristic_cost=heuristic_cost, action=state[1])
            self.frontier.push(node)

    def print_nodes(self):
        node = self.goal_node
        nodes = []
        while node is not None:
            nodes.append(node)
            node = node.parent
        for step, node in enumerate(reversed(nodes), start=1):
            print(f"Step: {step}")
            print(node.action)

In [ ]:
def heuristic0(curr_state):
    return 0
def heuristic1(curr_state):
    return sum(abs(i-3) + abs(j-3) for i in range(7) for j in range(7) if curr_state[i][j] == 1)

def heuristic2(curr_state):
    return sum(2 ** max(abs(i - 3), abs(j - 3)) for i in range(7) for j in range(7) if curr_state[i][j] == 1)


In [ ]:
t = 0
for i in range(5):
    agent = Player1(Environment(), heuristic2)
    t+=agent.run()

print("Average time", t/5)

print("Number of nodes explored:", len(agent.explored))
print("Number of nodes in frontier:", len(agent.frontier))

0.003085613250732422
0.0006940364837646484
0.0007064342498779297
0.0007033348083496094
0.0006604194641113281
Average time 0.0011699676513671875
Number of nodes explored: 1
Number of nodes in frontier: 0


In [ ]:
import time

class Player2:
    def __init__(self, env, heuristic):
        self.frontier = PriorityQueue()
        self.explored = {}
        self.env = env
        self.heuristic = heuristic
        self.start_state = self.env.get_start_state()
        self.goal_state = self.env.get_goal_state()
        self.goal_node = None

    def run(self):
        self.initialize_frontier()
        start_time = time.time()
        while not self.frontier.is_empty():
            current_node = self.frontier.pop()
            if self.is_explored(current_node):
                continue
            self.add_to_explored(current_node)
            if self.env.reached_goal(current_node.state):
                print("Reached goal!")
                self.goal_node = current_node
                break
            self.expand_frontier(current_node)
        end_time = time.time()
        print(end_time - start_time)
        return end_time - start_time

    def initialize_frontier(self):
        initial_node = Node(parent=None, state=self.start_state, path_cost=0, heuristic_cost=0)
        self.frontier.push(initial_node)

    def is_explored(self, node):
        return hash(node) in self.explored

    def add_to_explored(self, node):
        self.explored[hash(node)] = node

    def expand_frontier(self, current_node):
        next_states = self.env.get_next_states(current_node.state)
        for state in next_states:
            heuristic_cost = self.heuristic(state[0])
            node = Node(parent=current_node, state=state[0], path_cost=0, heuristic_cost=heuristic_cost, action=state[1])
            self.frontier.push(node)

    def print_nodes(self):
        node = self.goal_node
        nodes = []
        while node is not None:
            nodes.append(node)
            node = node.parent
        for step, node in enumerate(reversed(nodes), start=1):
            print(f"Step: {step}")
            print(node.action)

In [ ]:
t = 0
for i in range(5):
    agent = Player2(Environment(), heuristic2)
    t+=agent.run()

print("Average time", t/5)
print("Number of nodes explored:", len(agent.explored))
print("Number of nodes in frontier:", len(agent.frontier))

0.0008957386016845703
0.0007128715515136719
0.0018048286437988281
0.0010941028594970703
0.0006597042083740234
Average time 0.001033449172973633
Number of nodes explored: 1
Number of nodes in frontier: 0


Que 2: Uniform kSAT

In [ ]:
def generateInstance(n, k, m):
    vars = [chr(i + 65) for i in range(n)]
    clauses = []

    for _ in range(m):
        clause = []
        for _ in range(k):
            x = random.choice(vars)
            vars.remove(x)
            clause.append('~' + x if random.random() < 0.5 else x)
        clauses.append(' or '.join(clause))
        vars.extend(clause)

    problem = ' and '.join(f'({clause})' for clause in clauses)
    return f'(({problem}))'

for i in range(10):
    print(f"Problem {i+1}: {generateInstance(12, 3, 4)}")

Problem 1: (((L or K or ~G) and (~H or A or C) and (~~H or ~I or F) and (~~G or D or B)))
Problem 2: (((C or A or G) and (~H or ~K or ~D) and (F or ~L or ~C) and (E or ~C or ~D)))
Problem 3: (((~F or B or ~G) and (D or ~E or ~L) and (K or ~~F or ~~G) and (~J or D or H)))
Problem 4: (((B or ~C or ~D) and (~K or ~I or ~F) and (~~F or ~I or ~C) and (~~F or ~A or ~L)))
Problem 5: (((~K or F or E) and (~J or ~I or ~L) and (G or ~K or ~I) and (D or ~L or ~E)))
Problem 6: (((D or ~C or F) and (~I or A or B) and (~~I or ~L or ~A) and (~~~I or ~~C or B)))
Problem 7: (((D or ~B or ~H) and (J or ~B or ~E) and (~J or F or I) and (~~E or G or ~~B)))
Problem 8: (((~J or B or I) and (G or A or ~L) and (~J or ~H or I) and (F or ~G or I)))
Problem 9: (((~L or ~A or I) and (~H or ~E or ~G) and (~H or ~E or F) and (~~A or B or C)))
Problem 10: (((~K or I or H) and (~I or F or ~C) and (H or ~E or ~G) and (~~I or ~A or ~H)))


3SAT

In [ ]:
from dataclasses import dataclass, field
from typing import Any

@dataclass(order=True)
class PrioritizedItem:
    priority: int
    item: Any=field(compare=False)

In [ ]:
#Function to generate an instance of k-SAT problem
def generateInstance(n, k, m):
    vars = [chr(i + 65) for i in range(n)]
    problem_parts = []

    for _ in range(m):
        clause = []
        for _ in range(k):
            x = random.choice(vars)
            vars.remove(x)
            clause.append('~' + x if random.random() < 0.5 else x)
        problem_parts.append(' or '.join(clause))
        vars.extend(clause)

    problem = ' and '.join(f'({part})' for part in problem_parts)
    return f'(({problem}))'


In [ ]:
# Function to generate a random assignment of variables
def generateRandomAssignment(num_vars):
    return [random.randint(0, 1) for _ in range(num_vars)]

In [ ]:
# Function to evaluate the fitness of a given assignment of variables
def evaluate(assignment, k, variables, posOrNeg):
    fitness = 0
    clauseEval = 0

    for i, var in enumerate(variables):
        clauseEval |= assignment[var] if posOrNeg[i] == 'P' else 1 - assignment[var]
        if i % k == k - 1:
            fitness += clauseEval
            clauseEval = 0

    return fitness

Hill Climbing

In [ ]:
def hillClimbing(assignment, depth, k, variables, posOrNeg):
    for d in range(depth):
        currentFitness = evaluate(assignment, k, variables, posOrNeg)

        if currentFitness == len(variables):
            return assignment

        change = None
        for c in assignment:
            neighbour = assignment.copy()
            neighbour[c] = 1 - neighbour[c]

            neighbourFitness = evaluate(neighbour, k, variables, posOrNeg)
            if neighbourFitness > currentFitness:
                currentFitness = neighbourFitness
                change = c

        if change is not None:
            assignment[change] = 1 - assignment[change]

    return assignment

Beam Search (beam width must be less than n)

In [ ]:
#function to solve 3-SAT using Beam Search
def beamSearch(assignment, k, variables, posOrNeg, b, steps):
    beam = PriorityQueue()
    current = assignment
    beam.put(PrioritizedItem(-evaluate(current, k, variables, posOrNeg), assignment))

    for s in range(steps):
        if beam.empty():
            break

        state = beam.get()
        current = state.item
        x = -state.priority

        if x == len(variables):
            return current

        for c in current:
            neighbour = current.copy()
            neighbour[c] = 1 - neighbour[c]
            neighbourFitness = -evaluate(neighbour, k, variables, posOrNeg)

            if beam.qsize() < b or neighbourFitness > beam.queue[0].priority:
                if beam.qsize() == b:
                    beam.get()
                beam.put(PrioritizedItem(neighbourFitness, neighbour))

    return current

Variable-Neigbourhood-Descent (3 different neighbouring function)

In [ ]:
def neighbour1(assignment):
  c = random.choice(list(assignment))
  assignment[c] = 1 - assignment[c]
  return assignment

def neighbour2(assignment):
  c = random.choice(list(assignment))
  d = random.choice(list(assignment))
  while d == c:
    d = random.choice(list(assignment))
  x = assignment[c]
  assignment[c] = assignment[d]
  assignment[d] = x

  return assignment

def neighbour3(assignment):
  x = list(assignment.keys())[0]
  assignment[x] = 1 - assignment[x]
  return assignment

In [ ]:
def variableNeighbourhood(assignment, k, variables, posOrNeg, steps):
  s = 0
  current = assignment

  while s < steps:
    current = assignment
    x = evaluate(assignment, k, variables, posOrNeg)
    if x == len(variables):
      return current

    nbr1 = neighbour1(current.copy())
    nbr2 = neighbour2(current.copy())
    nbr3 = neighbour3(current.copy())

    fn1 = evaluate(nbr1, k, variables, posOrNeg)
    fn2 = evaluate(nbr2, k, variables, posOrNeg)
    fn3 = evaluate(nbr3, k, variables, posOrNeg)

    if max(fn1, fn2, fn3) > x:
      x = max(fn1, fn2, fn3)
      if x == fn1:
        current = nbr1
      elif x == fn2:
        current = nbr2
      else:
        x =  nbr3
    s += 1

  return current

In [ ]:
n = 25
k = 3
m = 1000
problem = generateInstance(n,k,m)
numVars = set()
variables = []
posOrNeg = []

prevNeg = False

for i in range(len(problem)):
  if problem[i] == '~':
    prevNeg = True
  elif ord(problem[i]) >= 65 and ord(problem[i]) <= 90:
    if prevNeg == True:
      posOrNeg.append('N')
      prevNeg = False
    else:
      posOrNeg.append('P')

    variables.append(problem[i])
    numVars.add(problem[i])


values = generateRandomAssignment(len(numVars))
startState = dict()

index = 0
for c in numVars:
  startState[c] = values[index]
  index += 1

print(startState)
print("Starting State Fitness: ", evaluate(startState, k, variables, posOrNeg))
solution  = hillClimbing(startState.copy(), 100, k, variables, posOrNeg)
print("Hill Climbing Solution Fitness: ", evaluate(solution, k, variables, posOrNeg))
solution = beamSearch(startState.copy(), k, variables, posOrNeg, 3, 1000)
print("Beam Search Solution Fitness (Beam-Width = 3): ", evaluate(solution, k, variables, posOrNeg))
solution = beamSearch(startState.copy(), k, variables, posOrNeg, 4, 1000)
print("Beam Search Solution Fitness (Beam-Width = 4): ", evaluate(solution, k, variables, posOrNeg))


print("Neighbour 1: ", neighbour1(startState.copy()))
print("Neighbour 2: ", neighbour2(startState.copy()))
print("Neighbour 3: ", neighbour3(startState.copy()))

solution = variableNeighbourhood(startState.copy(), k, variables, posOrNeg, 1000)
print("Variable-Neigbourhood-Descent Fitness: ", evaluate(solution, k, variables, posOrNeg))

{'S': 0, 'W': 0, 'M': 1, 'J': 1, 'K': 1, 'R': 0, 'Y': 0, 'D': 1, 'P': 1, 'L': 1, 'I': 1, 'E': 1, 'N': 0, 'T': 0, 'F': 0, 'G': 1, 'O': 0, 'X': 0, 'V': 0, 'H': 1, 'C': 1, 'A': 0, 'B': 0, 'Q': 0, 'U': 1}
Starting State Fitness:  900
Hill Climbing Solution Fitness:  1000
Beam Search Solution Fitness (Beam-Width = 3):  114
Beam Search Solution Fitness (Beam-Width = 4):  114
Neighbour 1:  {'S': 0, 'W': 0, 'M': 1, 'J': 1, 'K': 1, 'R': 0, 'Y': 0, 'D': 1, 'P': 1, 'L': 1, 'I': 0, 'E': 1, 'N': 0, 'T': 0, 'F': 0, 'G': 1, 'O': 0, 'X': 0, 'V': 0, 'H': 1, 'C': 1, 'A': 0, 'B': 0, 'Q': 0, 'U': 1}
Neighbour 2:  {'S': 0, 'W': 0, 'M': 0, 'J': 1, 'K': 1, 'R': 0, 'Y': 0, 'D': 1, 'P': 1, 'L': 1, 'I': 1, 'E': 1, 'N': 0, 'T': 0, 'F': 0, 'G': 1, 'O': 0, 'X': 0, 'V': 0, 'H': 1, 'C': 1, 'A': 0, 'B': 1, 'Q': 0, 'U': 1}
Neighbour 3:  {'S': 1, 'W': 0, 'M': 1, 'J': 1, 'K': 1, 'R': 0, 'Y': 0, 'D': 1, 'P': 1, 'L': 1, 'I': 1, 'E': 1, 'N': 0, 'T': 0, 'F': 0, 'G': 1, 'O': 0, 'X': 0, 'V': 0, 'H': 1, 'C': 1, 'A': 0, 'B': 0,